# 傾聴応答データベースの扱い方

- `alr.db`: 傾聴応答データをデータベース形式に変換したもの

この ipynb ファイルを参考に，`alr.db` から必要な情報を抽出する．
必ずしもこの ipynb ファイルをそのまま使用する必要はなく，使いやすい形に書き換えて良い．もちろん py ファイルを作って良い

## テーブルの定義

```
docs (
    id          INTEGER PRIMARY KEY AUTOINCREMENT, # 1-270（30人*9トピック）
    content     TEXT, # トピック単位の文章
    meta_info   BLOB, # id, 語り手（speaker），トピック（topic）
    sentence    BLOB, # 文単位の情報
    clause      BLOB, # 節単位の情報
    chunk       BLOB, # 文節単位の情報
    token       BLOB, # 形態素単位の情報
    response    BLOB # 応答の情報
)

sentence ->
[
    {
        'begin': contentにおける文の先頭の文字位置,
        'end': contentにおける文の末尾の文字位置,
        'starttime': 音声における文の発話開始時間,
        'endtime': 音声における文の発話終了時間
    }
]

clause ->
[
    {
        'begin': contentにおける節の先頭の文字位置,
        'end': contentにおける節の末尾の文字位置,
        'label': 節境界のラベル,
        'starttime': 音声における節の発話開始時間,
        'endtime': 音声における節の発話終了時間
    }
]

chunk ->
[
    {
        'begin': contentにおける文節の先頭の文字位置,
        'end': contentにおける文節の末尾の文字位置,
        'link': 係り先の文節番号
        'starttime': 音声における文節の発話開始時間,
        'endtime': 音声における文節の発話終了時間,
    }
]

token ->
[
    {
        'begin': contentにおける形態素の先頭の文字位置,
        'end': contentにおける形態素の末尾の文字位置,
        'lemma': 形態素の表層形,
        'POS': 形態素の品詞情報1,
        `POS2`: 形態素の品詞情報2,
        'starttime': 音声における形態素の発話開始時間,
        'endtime': 音声における形態素の発話終了時間,
    }
]

response ->
[
    {
        'begin': 発話時間をもとに対応付けたcontentにおける文節の先頭の文字位置,
        'end': 発話時間をもとに対応付けたcontentにおける文節の末尾の文字位置,
        'label': 応答種類のラベル,
        'lemma': 形態素の表層形,
        'starttime': 音声における形態素の発話開始時間,
        'endtime': 音声における形態素の発話終了時間,
        'listener': 聴き手のラベル
    }
]

```


## データベースを利用するための設定

- `connect` 関数で（同階層に置いてあると仮定している） `alr.db` に接続  

In [5]:
import json
import sqlite3

In [6]:
conn = None

In [9]:
def connect():
    global conn
    conn = sqlite3.connect('./alr.db')


def close():
    conn.close()


def get(doc_id, fl):
    row_ls = conn.execute(
        'SELECT {} FROM docs WHERE id = ?'.format(','.join(fl)),
        (doc_id,)).fetchone()
    row_dict = {}
    for key, value in zip(fl, row_ls):
        row_dict[key] = value
    return row_dict


def get_all_ids(limit, offset=0):
    return [record[0] for record in
            conn.execute(
        'SELECT id FROM docs LIMIT ? OFFSET ?',
        (limit, offset))]


def get_annotation(doc_id, name):
    row = conn.execute(
        'SELECT {0} FROM docs WHERE id = ?'.format(name),
        (doc_id,)).fetchone()
    if row[0] is not None:
        return json.loads(row[0])
    else:
        return []


## データベースを利用するためのclass

In [10]:
from typing import Dict, List, Tuple
import ast
from collections import defaultdict

In [11]:
class AlrAnnotation():
    """
    データベースを扱うためのクラス
    """

    def __init__(self, doc_id: str):
        self.doc_id = doc_id

    def get_content(self) -> str:
        """
        トピック単位のテキストを取得

        Args:
            なし
        Returns:
            row['content'] (str): トピック単位のテキスト
        """

        row = get(self.doc_id, ['content'])
        return row['content']

    def get_metainfo(self) -> Tuple[int, str, str]:
        """
        id, 語り手（speaker），トピック（topic）を取得

        Args:
            なし
        Returns:
            id (int), speaker (str), topic (str)
        """

        row = get(self.doc_id, ['id', 'meta_info'])
        meta_info = ast.literal_eval(row['meta_info'])

        return row['id'], meta_info['speaker'], meta_info['topic']

    def get_annotation(self, anno_type: str) -> List[str]:
        """
        指定されたタイプのannotationの情報を返す

        Args:
            anno_type (str)
            'sentence', 'clause', 'chunk', 'token', 'response'
        Returns:
            datastore.get_annotation(self.doc_id, anno_type)
        Example:
            sentences = get_annotation('sentence')
        """

        return get_annotation(self.doc_id, anno_type)

    def output_annotation(self):
        """
        content, tokens. chunks, clauses, sentences の情報を出力．
        本来，class には不必要
        """

        print('content:')
        text = self.get_content()
        print(text)
        print('tokens:')
        tokens = self.get_annotation('token')
        for token in tokens:
            print('  ', token['POS'], '\t', text[token['begin']:token['end']])
            print('  ', token['starttime'], token['endtime'])
        print('chunks:')
        chunks = self.get_annotation('chunk')
        for chunk in chunks:
            _, link = chunk['link']
            print(
                '  ', chunk['starttime'], chunk['endtime'],
                text[chunk['begin']:chunk['end']])
            if link != -1:
                parent = chunks[link]
                print('\t-->', text[parent['begin']:parent['end']])
            else:
                print('\t-->', 'None')
        print('clauses:')
        clauses = self.get_annotation('clause')
        for clause in clauses:
            print(
                '  ', text[clause['begin']:clause['end']],
                '\t', clause['label'])
            print('  ', clause['starttime'], clause['endtime'])
        print('sentences:')
        sentences = self.get_annotation('sentence')
        for sent in sentences:
            print('  ', text[sent['begin']:sent['end']])
            print('  ', sent['starttime'], sent['endtime'])

    def output_response(self):
        """
        発話時間の制約を満たす傾聴応答の情報を出力．
        サンプルであり，本来，class には不必要
        """

        text = self.get_content()
        unit_type = 'clause'
        units = self.get_annotation(unit_type)

        responses = self.get_annotation('response')

        for unit in units:
            print(
                text[unit['begin']:unit['end']],
                '\t', unit['starttime'], '\t', unit['endtime'])

            for resp in responses:
                # 言語単位の発話時間内に発話が開始されている応答を表示
                if unit['starttime'] <= resp['starttime'] < unit['endtime']:
                    print(f"  {resp['listener']}, {resp['lemma']}, {resp['label']}, {resp['starttime']}, {resp['endtime']}")
                    # print(f"{text[resp['begin']:resp['end']]}, {resp['label']}, {resp['listener']}")

    def get_response(self) -> Dict[str, list]:
        """
        トピック単位の傾聴応答の情報を取得．
        目的とする処理のために情報を組み替える．

        Args:
            なし
        Returns:
            new_responses
        """

        new_responses = defaultdict(list)
        listener = 'o'  # database に 'o', 'a', 'b', ... の順番で格納しているので'o' から開始
        responses = self.get_annotation('response')
        for resp in responses:
            if listener != resp['listener']:
                pass
                # break  # 一人目である o さんの応答だけ取るなら break

            # all_responses_lemma[resp['label']].append(resp['lemma'])  # ラベルごとの傾聴応答文字列を dictionary で保管 -> TODO: 対応が必要

            # 対応付いた文節の位置情報
            duration = str(resp['begin']) + '-' + str(resp['end'])

            # 文節の位置情報をキーとしたリストの辞書を作成
            # output_for_classification, output_for_t5 ならこっち
            # Example: back-channel:うわー
            new_responses[duration].append(resp['label']+':'+resp['lemma'])
            # output_for_generationtimingならこっち
            # new_responses[duration].append(resp['listener'])

            # 情報の更新
            listener = resp['listener']

        return new_responses


## データベースの利用

### データベース接続

In [12]:
connect()

### id の出力

In [13]:
print(get_all_ids(-1))

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 22

### インスタンスの作成

- id を引数に

In [14]:
alr_anno = AlrAnnotation(1)  # id を引数にしてインスタンスを作成

### content（トピック単位の文章）の出力

- ついでに meta_info も出力

In [15]:
# meta_info の出力
id, speaker, topic = alr_anno.get_metainfo()
print(id, speaker, topic)
# 1 01 1

# content の出力
print(alr_anno.get_content())
# 子供がですね三人女の子ばっかしなんですけど産まれてことですね。
# でさらに今現在ではですね孫もおりますのでそういう意味ではほかの友達なんかに比べて非常に恵まれてるということで今でも感謝し嬉しくなったと思います。
# あとは大学を卒業してからですね一番当時では難しい鉄鋼メーカーに就職することができたんですね。
# でこれがまずとっかかりで非常に嬉しくてですね今まで経験しなかったことこれ仕事の面でもそれから人の面でもそういうことがありまして今現在でもですねそういう人達との交流があるからこそ今の私があるということでそういう意味では非常に感謝しております。
# ですから今でもいい企業に入ったなということを今でも十分感じしております。

1 01 1
子供がですね三人女の子ばっかしなんですけど産まれてことですね。
でさらに今現在ではですね孫もおりますのでそういう意味ではほかの友達なんかに比べて非常に恵まれてるということで今でも感謝し嬉しくなったと思います。
あとは大学を卒業してからですね一番当時では難しい鉄鋼メーカーに就職することができたんですね。
でこれがまずとっかかりで非常に嬉しくてですね今まで経験しなかったことこれ仕事の面でもそれから人の面でもそういうことがありまして今現在でもですねそういう人達との交流があるからこそ今の私があるということでそういう意味では非常に感謝しております。
ですから今でもいい企業に入ったなということを今でも十分感じしております。



### 各 annotation の出力

- sentence
- clause
- chunk
- token

In [10]:
# annotation の出力
alr_anno.output_annotation()

content:
子供がですね三人女の子ばっかしなんですけど産まれてことですね。
でさらに今現在ではですね孫もおりますのでそういう意味ではほかの友達なんかに比べて非常に恵まれてるということで今でも感謝し嬉しくなったと思います。
あとは大学を卒業してからですね一番当時では難しい鉄鋼メーカーに就職することができたんですね。
でこれがまずとっかかりで非常に嬉しくてですね今まで経験しなかったことこれ仕事の面でもそれから人の面でもそういうことがありまして今現在でもですねそういう人達との交流があるからこそ今の私があるということでそういう意味では非常に感謝しております。
ですから今でもいい企業に入ったなということを今でも十分感じしております。

tokens:
   名詞 	 子供
   76.75 77.27
   助詞 	 が
   77.27 77.45
   助動詞 	 です
   77.45 77.72
   助詞 	 ね
   77.72 77.83
   名詞 	 三
   77.91 78.20
   名詞 	 人
   78.20 78.62
   名詞 	 女の子
   78.89 79.31
   助詞 	 ばっか
   79.31 79.58
   動詞 	 しな
   79.58 79.84
   助動詞 	 ん
   79.84 79.90
   助動詞 	 です
   79.90 80.09
   助詞 	 けど
   80.09 80.30
   動詞 	 産まれ
   80.35 80.76
   助詞 	 て
   80.76 81.06
   名詞 	 こと
   82.19 82.45
   助動詞 	 です
   82.45 82.75
   助詞 	 ね
   82.75 82.90
   記号 	 。
   None None
   接続詞 	 で
   83.22 83.57
   副詞 	 さらに
   83.78 84.45
   接頭詞 	 今
   84.45 84.64
   名詞 	 現在
   84.64 85.12
   助詞 	 で
   85.12 85.23
   助詞 	 は
   85.23 85.37
   助動詞 	 です
   85.37 85.60
   助詞 	 ね
   85.60 85.70


### 応答情報の出力

- 語りのある単位（sentence, clause, chunk, token）の発話開始から終了までに発話が開始されている応答情報を出力

In [11]:
# annotation の出力
alr_anno.output_response()

子供がですね 	 76.75 	 77.83
三人女の子ばっかしなんですけど 	 77.91 	 80.30
  o, ええ, back-channel, 78.28, 78.46
  a, はい, back-channel, 78.04, 78.28
  c, はい, back-channel, 78.12, 78.54
  f, ええ, back-channel, 78.07, 78.39
  g, はい, back-channel, 78.32, 78.79
  h, はい, back-channel, 78.01, 78.4
  i, ああ, other, 78.19, 78.45
  i, うん, back-channel, 79.65, 79.76
  i, うーんうんうんうんうん, back-channel, 79.76, 81.23
  j, うーん, back-channel, 78.3, 78.83
産まれて 	 80.35 	 81.06
  b, はい, back-channel, 80.66, 80.86
  c, ええ, back-channel, 80.43, 80.63
  f, ええ, back-channel, 80.53, 80.92
  g, あ, admiration, 80.43, 80.58
  g, そうですか, admiration, 80.84, 81.77
  h, そうなんですか, admiration, 80.55, 81.42
  j, うんうん, back-channel, 80.83, 81.39
ことですね。 	 82.19 	 82.90
で 	 83.22 	 83.57
  o, はい, back-channel, 83.34, 83.85
  c, あ, other, 83.42, 83.51
  f, ええええ, back-channel, 83.24, 83.67
  j, うん, back-channel, 83.31, 83.65
さらに 	 83.78 	 84.45
今現在ではですね 	 84.45 	 85.70
  o, ええ, back-channel, 84.71, 84.83
  g, ええ, back-channel, 85.53, 85.89
  h, ええ, bac

### 【参考】語り音声の再生

In [16]:
import subprocess

In [17]:
narrative_mp3_list = {
    "01": "01_Interview_M8201.mp3",
    "02": "02_Interview_F7502.mp3",
    "03": "03_Interview_M7703.mp3",
    "04": "04_Interview_F8204.mp3",
    "05": "05_Interview_F7205.mp3",
    "06": "06_Interview_M7306.mp3",
    "07": "07_Interview_M7407.mp3",
    "08": "08_Interview_F8208.mp3",
    "09": "09_Interview_F7109.mp3",
    "10": "10_Interview_F7210.mp3",
    "11": "11_Interview_F7311.mp3",
    "12": "12_Interview_M7012.mp3",
    "13": "13_Interview_F7513.mp3",
    "14": "14_Interview_M7514.mp3",
    "15": "15_Interview_F7815.mp3",
    "16": "16_Interview_M7816.mp3",
    "17": "17_Interview_M7117.mp3",
    "18": "18_Interview_F7718.mp3",
    "19": "19_Interview_M7919.mp3",
    "20": "20_Interview_F8020.mp3",
    "21": "21_Interview_M8521.mp3",
    "22": "22_Interview_F8622.mp3",
    "23": "23_Interview_M7723.mp3",
    "24": "24_Interview_M7824.mp3",
    "25": "25_Interview_M7425.mp3",
    "26": "26_Interview_M7226.mp3",
    "27": "27_Interview_M7727.mp3",
    "28": "28_Interview_F7628.mp3",
    "29": "29_Interview_F8129.mp3",
    "30": "30_Interview_F7530.mp3",
}

In [18]:
def play_narrative_mp3(alr_anno: AlrAnnotation, anno_type: str = 'sentence'):
    """
    ある言語単位の語りの音声を再生．

    Args:
        alr_anno (AlrAnnotation)
    Returns:
        なし
    Example:
    """

    units = alr_anno.get_annotation(anno_type)
    for unit in units:
        print(unit)

        _, speaker, _ = alr_anno.get_metainfo()

        wavfile = f'/Users/masaki/Kenkyu/Elderly-persons_interview/Attentive_Listening_Responses_Corpus/audio/{speaker}/{narrative_mp3_list[speaker]}'

        start = unit['starttime']
        duration = float(unit['endtime']) - float(unit['starttime'])

        cmd = f'play {wavfile} trim {start} {duration}'
        print(cmd)
        subprocess.call(cmd, shell=True)


In [ ]:
play_narrative_mp3(alr_anno, anno_type='sentence')

### データベース切断

In [20]:
close()

## 【メモ】学習，開発，テストデータの区分

- docs の id で指定
- これまでに使用されてきた区分だが，変更しても良い
- 通常，学習，開発，テストデータは全データを 8:1:1 や 6:2:2 に分割して作成される

In [13]:
train_id = [
    1, 2, 3, 4, 5, 6, 7, 10, 11, 12, 13, 14, 15, 16,
    19, 20, 21, 22, 23, 24, 25, 28, 29, 30, 31, 32, 33, 34,
    37, 38, 39, 40, 41, 42, 43, 46, 47, 48, 49, 50, 51, 52,
    55, 56, 57, 58, 59, 60, 61, 64, 65, 66, 67, 68, 69, 70,
    73, 74, 75, 76, 77, 78, 79, 82, 83, 84, 85, 86, 87, 88,
    91, 92, 93, 94, 95, 96, 97, 100, 101, 102, 103, 104, 105, 106,
    109, 110, 111, 112, 113, 114, 115, 118, 119, 120, 121, 122, 123, 124,
    127, 128, 129, 130, 131, 132, 133, 136, 137, 138, 139, 140, 141, 142,
    145, 146, 147, 148, 149, 150, 151, 154, 155, 156, 157, 158, 159, 160,
    163, 164, 165, 166, 167, 168, 169, 172, 173, 174, 175, 176, 177, 178,
    181, 182, 183, 184, 185, 186, 187, 190, 191, 192, 193, 194, 195, 196,
    199, 200, 201, 202, 203, 204, 205, 208, 209, 210, 211, 212, 213, 214
]

dev_id = [
    8, 17, 26, 35, 44, 53, 62, 71, 80, 89, 98,
    107, 116, 125, 134, 143, 152, 161, 170, 179, 188, 197,
    206, 215, 217, 218, 219, 220, 221, 222, 223, 224, 226, 227, 228, 229,
    230, 231, 232, 233, 235, 236, 237, 238, 239, 240, 241, 242, 251, 260, 269
]

test_id = [
    9, 18, 27, 36, 45, 54, 63, 72, 81, 90, 99,
    108, 117, 126, 135, 144, 153, 162, 171, 180, 189, 198,
    207, 216, 225, 234, 243, 244, 245, 246, 247, 248, 249,
    250, 252, 253, 254, 255, 256, 257, 258, 259,
    261, 262, 263, 264, 265, 266, 267, 268, 270
]